<a href="https://colab.research.google.com/github/monteroanibal/diplomado_GEOIA_IGAC2025/blob/main/unidad_1_6_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook (I)

---

# 1. Cargue de librerías

Para importar shapefiles como _GeoDataFrames_.

In [1]:
import geopandas as gpd
import pandas as pd; pd.set_option('display.float_format', lambda x: '%.1f' % x)

Para generar los mapas interactivos.

In [2]:
%%capture
!pip install pydeck
import pydeck as pdk

Para hacer cambios en los formatos de los datos.

In [3]:
import json
import numpy as np

Para cronometrar el tiempo de ejecución de los renders de los mapas interactivos.

In [4]:
from time import time

# 2. Lectura y manipulación de datos vectoriales

Se descargan los shapefiles que están publicados en un Github y se descomprimen en la sesión actual.

In [5]:
%%capture
!wget https://github.com/monteroanibal/diplomado_GEOIA_IGAC2025/raw/refs/heads/main/Insumos/Unidad_1_6_Datos/shapefiles.zip
!unzip -o /content/shapefiles.zip
!rm /content/shapefiles.zip

Se descargan los datos que se encuentran publicados en un GitHub y se descomprimen en la sesión actual.

In [6]:
mpio_chia = gpd.read_file("/content/shapefiles/Mpio_Chia/Mpio_Chia.shp")
mzn_cnpv_2018 = gpd.read_file("/content/shapefiles/Censo_DANE_2018/Censo_DANE_2018.shp")
construc_2018 = gpd.read_file("/content/shapefiles/Construccion/Construccion.shp")

Se toman unicamente los campos necesarios para construir los mapas interactivos.

In [7]:
mzn_cnpv_2018 = mzn_cnpv_2018[['TVIVIENDA','TP16_HOG','TP27_PERSO','geometry']]

In [8]:
construc_2018 = construc_2018[['tipo_const','tipo_domin','numero_pis','geometry']]
construc_2018['area_calc'] = construc_2018['geometry'].to_crs(epsg=3116).area

Para realizar una extrusión en el mapa, es necesario escalar uno de los atributos numéricos de la capa de información entre 0 y 1.

In [9]:
mzn_cnpv_2018[['personas_norm']] = (mzn_cnpv_2018[['TP27_PERSO']]-mzn_cnpv_2018[['TP27_PERSO']].min())/(mzn_cnpv_2018[['TP27_PERSO']].max()-mzn_cnpv_2018[['TP27_PERSO']].min())

In [10]:
construc_2018[['num_pis_norm']] = (construc_2018[['numero_pis']]-construc_2018[['numero_pis']].min())/(construc_2018[['numero_pis']].max()-construc_2018[['numero_pis']].min())

Para que la interactividad nos muestre los atributos de cada manzana y edificio en la capa de información de forma resumida, se convierten los valores tipo _float_ a valores tipo _int_.

In [11]:
mzn_cnpv_2018[['TVIVIENDA','TP16_HOG','TP27_PERSO']] = mzn_cnpv_2018[['TVIVIENDA','TP16_HOG','TP27_PERSO']].map(np.int64)

In [12]:
construc_2018[['numero_pis','area_calc']] = construc_2018[['numero_pis','area_calc']].map(np.int64)

La librería requiere que la capa de información sea convertida a un formato de datos tipo _json_. Se verifican los valores de los campos en la primera entrada de la capa de información.

In [13]:
mzn_cnpv_2018_yeison = json.loads(mzn_cnpv_2018.to_json())
mzn_cnpv_2018_yeison["features"][0]["properties"]

{'TVIVIENDA': 48,
 'TP16_HOG': 63,
 'TP27_PERSO': 162,
 'personas_norm': 0.0739051094890511}

In [14]:
construc_2018_yeison = json.loads(construc_2018.to_json())
construc_2018_yeison["features"][0]["properties"]

{'tipo_const': 'CONVENCIONAL',
 'tipo_domin': 'PRIVADO',
 'numero_pis': 2,
 'area_calc': 155,
 'num_pis_norm': 0.14285714285714285}

# 3. Despliegue de mapa interactivo con _pydeck_: manzanas

Se configura la visualización para que se ubique sobre el municipio de Chía.

In [15]:
init_view_state = pdk.ViewState(
  longitude = mpio_chia.LONGITUD.tolist()[0],
  latitude = mpio_chia.LATITUD.tolist()[0],
  zoom=11.5,
  max_zoom=16,
  pitch=30,
  bearing=0
)

Se configura la capa de información para ser visualizada.

In [16]:
r = "100"
g = "255"
b = "properties.personas_norm * 255"
fill = f"[{r},{g},{b}]"

capa_manzanas = pdk.Layer(
  'GeoJsonLayer',
  mzn_cnpv_2018_yeison,
  opacity=0.8,
  pickable=True,
  stroked=True,
  filled=True,
  extruded=True,
  wireframe=True,
  auto_highlight=True,
  get_elevation="properties.personas_norm * 10",
  elevation_scale=100,
  get_fill_color=fill,
)

Se configura la información desplegada durante la navegación con el cursor.

In [17]:
tooltip = {"text": "Núm. viviendas: {TVIVIENDA}\nNúm. hogares: {TP16_HOG}\nNúm. personas: {TP27_PERSO}"}

Se ejecuta el render de la visualización y se mide el tiempo que está tardando en hacer la tarea.

In [18]:
empieza = time()

In [ ]:
r = pdk.Deck(
  layers=[capa_manzanas],
  initial_view_state=init_view_state,
  tooltip=tooltip,
  map_style=pdk.map_styles.DARK,
)
r.to_html()

In [20]:
print('Tiempo de renderización en segundos: ', round(time()-empieza,3))

Tiempo de renderización en segundos:  0.468


# 4. Despliegue de mapa interactivo con _pydeck_: construcciones

Se configura la visualización para que se ubique sobre el municipio de Chía.

In [21]:
init_view_state = pdk.ViewState(
  longitude = mpio_chia.LONGITUD.tolist()[0],
  latitude = mpio_chia.LATITUD.tolist()[0],
  zoom=12,
  max_zoom=18,
  pitch=60,
  bearing=180,
)

Se configura la capa de información para ser visualizada.

In [22]:
r = "properties.num_pis_norm * 30"
g = "30"
b = "220"
fill = f"[{r},{g},{b}]"

capa_construcciones = pdk.Layer(
  'GeoJsonLayer',
  construc_2018_yeison,
  opacity=0.05,
  pickable=True,
  stroked=True,
  filled=True,
  extruded=True,
  wireframe=True,
  auto_highlight=True,
  get_elevation="0.05 + ( properties.num_pis_norm * 3 / 4 )",
  elevation_scale=100,
  get_fill_color=fill,
)

Se configura la información desplegada durante la navegación con el cursor.

In [23]:
tooltip = {"text": "Núm. pisos: {numero_pis}\nÁrea construida: {area_calc}\nTipo de construcción: {tipo_const}\nTipo de dominio: {tipo_domin}"}

Se ejecuta el render de la visualización.

In [24]:
empieza = time()

In [ ]:
r = pdk.Deck(
  layers=[capa_construcciones],
  initial_view_state=init_view_state,
  tooltip=tooltip,
  map_style=pdk.map_styles.ROAD,
)
r.to_html()

In [26]:
print('Tiempo de renderización en segundos: ', round(time()-empieza,3))

Tiempo de renderización en segundos:  6.098


**<center>Un consejo**</center>

_pydeck_ no indica errores durante el desarrollo del código ni cuando los datos de la capa no están formateados correctamente. Esto puede resultar en tiempos de renderizado largos, solo para descubrir que tienes un mapa vacío. Para solucionar esto, es mejor optar por trabajar con el encabezado o una pequeña muestra de los datos hasta que el mapa funcione correctamente.